In [1]:
import pandas as pd
from sklearn.decomposition import PCA
from sentence_transformers import SentenceTransformer, util
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import random
from scipy.stats import pearsonr
from transformers import pipeline
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
model = SentenceTransformer('all-MiniLM-L6-v2')

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")


In [4]:
file_name = "09-07-2017 audio_6 APCNV-2016-565_labeled.csv"
meeting = pd.read_csv(file_name)

In [5]:
sentences = meeting["text"].tolist()
labels = meeting["label"].tolist()
for_labels = [1 if label == 1 else 0 for label in labels]
neg_labels = [1 if label == -1 else 0 for label in labels]



In [6]:
sentences_embeddings = model.encode(sentences)

In [8]:
def find_similar_sentences(topic_sentences, sentence_embeddings, sentences, threshold=0.6):
    similar_sentences = []
    topic_sentence_embedding = model.encode(topic_sentences)
    
    for i, sentence_embedding in enumerate(sentence_embeddings):
        cos_similarity_ls = [
            util.pytorch_cos_sim(sentence_embedding, topic_embedding).item()
            for topic_embedding in topic_sentence_embedding
        ]
        
        if any(val > threshold for val in cos_similarity_ls):
            similar_sentences.append(1)
        else:
            similar_sentences.append(0)
    
    return similar_sentences


In [9]:
topic_sentence = [
  "commissioner vote",
  "official vote",
]

threshold = 0.3
similar_sentences = find_similar_sentences(topic_sentence, sentences_embeddings, sentences, threshold)

In [10]:
similar_sentences

[0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0]

In [20]:
hypothesis = ["This person supports the measure", "This person opposes the measure", "This person neither supports nor opposes the measure"]

threshold_classifier = 0.7
# for_prob = []
# against_prob = []
# neither_prob = []
results_ls = []
for i, sentence in enumerate(sentences):
  if similar_sentences[i] == 1:
    # for_prob.append(0)
    # against_prob.append(0)
    # neither_prob.append(1)
    results_ls.append([0, 0, 1])
  else:
    results = classifier(sentence, candidate_labels=hypothesis)

    # for_prob.append(results['scores'][0])
    # against_prob.append(results['scores'][1])
    # neither_prob.append(results['scores'][2])
    results_ls.append(results['scores'])


  # threshold = 0.7  # You can adjust this threshold
  # predicted_labels = [1 if result['scores'][0] > threshold_classifier else 0 for result in results]
  
  # predicted_prob_labels = [round(result['scores'][0], 2) for result in results]


In [21]:
results

{'sequence': '452',
 'labels': ['This person supports the measure',
  'This person opposes the measure',
  'This person neither supports nor opposes the measure'],
 'scores': [0.4400576651096344, 0.39921900629997253, 0.1607232689857483]}

In [23]:
neither_prob

[0.02843320555984974,
 0.012711365707218647,
 1,
 0.12903077900409698,
 1,
 0.2233685851097107,
 0.1481534093618393,
 0.12724286317825317,
 0.2233685851097107,
 0.08027182519435883,
 0.2233685851097107,
 0.17576928436756134,
 0.21160142123699188,
 0.07237844169139862,
 0.1266041249036789,
 0.129686638712883,
 0.16279146075248718,
 0.09606599062681198,
 0.02881716750562191,
 0.04284099116921425,
 0.12782692909240723,
 0.045281752943992615,
 0.08274385333061218,
 0.16851501166820526,
 0.04607558622956276,
 0.1410328894853592,
 0.2233685851097107,
 0.21731677651405334,
 0.2465946227312088,
 0.06886359304189682,
 0.24223387241363525,
 0.021249817684292793,
 0.12749657034873962,
 0.10303936153650284,
 0.11493764072656631,
 0.012998492456972599,
 0.02374492958188057,
 0.015950163826346397,
 1,
 0.016785575076937675,
 0.015918076038360596,
 0.2174363136291504,
 0.03287072479724884,
 0.13185814023017883,
 1,
 0.016133559867739677,
 1,
 0.2241649031639099,
 1,
 0.06113816425204277,
 0.026666229